# Random Forest

~a.k.a 묻지마모델~

Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks that operates by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees.

(출처: https://en.wikipedia.org/wiki/Random_forest)

To quote from The Elements of Statistical Learning, <b>"Trees have one aspect that prevents them from being the ideal tool for predictive learning, namely inaccuracy."</b>

$\rightarrow$ In other words, they work great with the data used to create them, but <b>they are not flexible when it comes to classifying new samples.</b>

<span style="color:blue">Step 1. Create a bootstrap dataset</span>

<b>Original Dataset</b>
<img src="img/RF_01_class.PNG">

Original dataset과 같은 크기의 Bootstrap dataset을 만들기 위해서 <b>임의로(Ramdomly) Original dataset에서 복원추출</b>을 할 것이다.

** Allow duplicate entries in the bootstrapped dataset.

<b>Bootstrapped Dataset</b>
<img src="img/RF_02_class.PNG">

<span style="color:blue">Step 2. Create a decision tree using the bootstrapped dataset, but only use a random subset of variables (or columns) at each step.</span>

Tree를 분할(split) 할 때 네 개의 독립변수(independent variable)를 모두 이용하지 않고 임의로 두 개만 선택을 한다.

"좋은 혈액순환 여부"와 "동맥경화 여부"를 선택되었고, "동맥경화 여부"가 분할에 가장 적합한 변수였다고 하면 해당 변수로 Tree를 분할 한다.

그러면 그 다음 또 다시 네 개의 독립변수 중 두 개를 다시 임의로 골라서 분할에 적합한 변수를 선택한다.

이 과정을 반복되면서 다양한 모양의 Tree를 키우게 된다.



최종적으로 Tree들을 만들게 되면 Classification을 할 데이터를 각 Tree에 넣어 결과를 받아본다. 가장 많은 Tree가 분류한 결과(Class)로 분류가 된다.

#### Out-Of-Bag(OOB) dataset
Bootstrap dataset에 들어가지 못한 데이터(entries)를 의미한다.

Tree를 구성하는데 쓰이지 않았기 때문에 OOB dataset으로 Random Forest의 성능을 평가할 수 있다.

OOB 데이터 중 잘못 분류된 비율을 <b>Out-Of-Bag Error</b>라고 한다.

다시 정리하자면,<br>
방금 전에는 한 step마다 독립변수 두 개를 임의로 추출하여 그 중 하나를 선택하는 것으로 결정하였지만 OOB Error를 구할 수 있게 됨으로써 세 개를 임의로 추출하는 방법과 비교하여 어느 것이 더 정확한 Random Forest를 만들 수 있는지 알 수 있게 되었다.

# Missing data and sample clustering

Random Forests consider 2 types of missing data...

1) Missing data in the original dataset used to create the random forest

2) Missing data in a new sample that you want to categorize

## #1 Missing data in the original dataset used to create the random forest

<b>Filled-in Missing Values</b>
<img src="img/RF_03_class.PNG">
물론 initial guess는 틀릴 수 있다.

이제 이 가정에 대한 평가를 진행할 것이다.

먼저 결측치가 보완된 dataset으로 Random Forest를 만든다.<br>
그리고 모든 데이터를 생성한 모든 Tree에 넣어 결과를 받아본다.

일단 <b>첫 번째 Tree</b>에 넣어본 결과....

<img src="img/RF_04_class.PNG">

데이터가 다음과 같이 분류되었다.<br>
특히 initial guess가 들어간 Entry 4는 Entry 3와 같은 Class로 분류되었다.

즉, 두 데이터(entry)는 비슷하다는 뜻이다.

이러한 결과들을 Proximity Matrix에 계속해서 기록할 것이다.<br>
Proximity Matrix는 아래와 같이 생겼다.

<img src="img/RF_05_class.PNG">
Entry 3과 4가 같은 클래스로 분류된 것을 표현하였다.

<b> 두 번째 Tree </b>로 분류한 결과 Entry 2,3,4가 같은 클래스로 분류되었다면, Proximity Matrix는 아래와 같이 업데이트 된다.

<img src="img/RF_06_class.PNG">

<b>총 10개의 Tree를 생성했다고 하였을 때</b> 최종적인 Proximity Matrix는 다음과 같다.

<img src="img/RF_07_class.PNG">

이제 Proximity Matrix를 가지고 결측치에 대한 더 나은 guess를 할 수 있다.

<b>Categorical Data</b>
<img src="img/RF_08_class.PNG">

<b>동맥경화 여부</b>에서 Proximity Matrix의 Value를 Weight로 사용하여<br> Entry 4의 결측치가 "Yes"와 "No" 중 어디에 가까운지 계산할 것이다.

"Yes"는 Entry 1,2,3 중 두 번째만이 가지고 있다.<br>
따라서, <b>The frequency of "Yes" = $1 \over 3$</b>

"No"는 Entry 1,2,3 중 첫 번째와 세 번째가 가지고 있다.<br>
따라서, <b>The frequency of "No" = $2 \over 3$</b>

이 때 Entry 4에서 Entry 2가 유사한 정도는 <span style="color:green">0.1</span>이다.<br>
따라서, <b>The weight for "Yes" = ${1 \over 3} * {0.1 \over {0.1 + 0.1 + 0.8}} = 0.03$</b>

한편 Entry 4에서 Entry 1과 3이 유사한 정도는 <span style="color:red">0.1</span>과 <span style="color:red">0.8</span>이다.<br>
따라서, <b>The weight for "No" = ${2 \over 3} * {{0.1+0.8 }\over {0.1 + 0.1 + 0.8}} = 0.6$</b>

$\rightarrow$ 결과적으로 "No"가 훨씬 더 높은 weighted frequency를 가지고 있기 때문에 Entry 4의 동맥경화 여부 결측치는 No로 guess를 한다.

<b>Numerical Data</b>
<img src="img/RF_09_class.PNG">

<b>체중</b>에서 Proximity Matrix의 Value를 Weight로 사용하여<br> Entry 4의 결측치를 Entry 1,2,3의 Weighted Average로 guess할 것이다.

따라서, <b>The weighted average for Entry 4 = $(125 * 0.1) + (180 * 0.1) + (210 * 0.8) = 198.5$</b>

다음과 같이 업데이트 된 결측치를 갖게 된다.
<img src="img/RF_10_class.PNG">


따라서 Random Forest 모델을 만들고 데이터를 input하여 결측치에 대한 guess를 계속해서 업데이트 한다.

이 작업을 결측치 value가 <u>converge(i.e. no longer change each time we recalculate)</u>할 때까지 계속한다.

##  #2 Missing data in a new sample that you want to categorize

이미 Random Forest 모델을 만든 상황에서 새로운 데이터를 넣어 예측하려고 한다.
<img src="img/RF_11_class.PNG">
그런데 위와 같이 "동맥경화 여부"에 결측치가 있기 때문에 모델을 적용하면 에러가 발생한다. 따라서 이 부분을 해결해줄 필요가 있다.

우선 아래와 같이 두 가지 종류의 데이터를 만든다.
<img src="img/RF_12_class.PNG">
그 다음 \#1에서 사용한 것과 같은 iterative method를 사용한다.

Random Forest를 구성하는 Tree들이<br>
"동맥경화 여부"가 Yes일 때 "심장질환 여부"를 Yes로 대답한 경우가 3번<br>
"동맥경화 여부"가 No일 때 "심장질환 여부"를 No로 대답한 경우가 1번<br>이라고 할 때,

"동맥경화 여부"가 Yes일 때 더 많이 맞추었기 때문에 해당 값으로 대체(imputation)한다.


## 참고.

1. <b>B</b>ootstrapping the data plus using the <b>agg</b>regate to make a decision is called <b>"Bagging"</b>

2. 비율화한 Proximity Matrix의 값들을 1에서 빼면 Distance Matrix로 바꿀 수 있다. 그리고 이를 시각화하면 Heatmap을 그리게 된다.

출처:::

https://www.youtube.com/watch?v=nyxTdL_4Q-Q